In [7]:
import copy
import cv2
import pandas as pd
import numpy as np
import math
import os
from joblib import dump, load

In [8]:
def get_results_dict():
  results_dict = {
      'HFL' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'HFR' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'LNF' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'HNL' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'HNR' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'LNN' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'LFI' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'LFO' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'LNI' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'LNO' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'SFL' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'SFR' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'SNL' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'SNR' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'SLFI' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'SLFO' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'SLNI' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'SLNO' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'after_end' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'let' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'dummy' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'hit_near' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'hit_far' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'land_near' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'land_far' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'serve_near' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
      'serve_far' : {'TP':{'count':0,'dist_arr':[]},'FP':{'count':0,'dist_arr':[]},'FN':{'count':0,'dist_arr':[]}},
  }
  return results_dict

def get_count_dict():
  results_dict = {
      'HFL' : {'count':0,'after_end':0},
      'HFR' : {'count':0,'after_end':0},
      'LNF' : {'count':0,'after_end':0},
      'HNL' : {'count':0,'after_end':0},
      'HNR' : {'count':0,'after_end':0},
      'LNN' : {'count':0,'after_end':0},
      'LFI' : {'count':0,'after_end':0},
      'LFO' : {'count':0,'after_end':0},
      'LNI' : {'count':0,'after_end':0},
      'LNO' : {'count':0,'after_end':0},
      'SFL' : {'count':0,'after_end':0},
      'SFR' : {'count':0,'after_end':0},
      'SNL' : {'count':0,'after_end':0},
      'SNR' :{'count':0,'after_end':0},
      'SLFI' : {'count':0,'after_end':0},
      'SLFO' : {'count':0,'after_end':0},
      'SLNI' : {'count':0,'after_end':0},
      'SLNO' : {'count':0,'after_end':0},
      'after_end' : {'count':0,'after_end':0},
      'let' :{'count':0,'after_end':0},
      'dummy' : {'count':0,'after_end':0},
  }
  return results_dict


def editDistance(str1, str2, m, n):
    #code from: https://www.geeksforgeeks.org/edit-distance-dp-5/
    # If first string is empty, the only option is to
    # insert all characters of second string into first
    #print(f"ED m {m} n {n}")
    if m == 0:
        return n
  
    # If second string is empty, the only option is to
    # remove all characters of first string
    if n == 0:
        return m
  
    # If last characters of two strings are same, nothing
    # much to do. Ignore last characters and get count for
    # remaining strings.
    if str1[m-1] == str2[n-1]:
        return editDistance(str1, str2, m-1, n-1)
  
    # If last characters are not same, consider all three
    # operations on last character of first string, recursively
    # compute minimum cost for all three operations and take
    # minimum of three values.
    return 1 + min(editDistance(str1, str2, m, n-1),    # Insert
                   editDistance(str1, str2, m-1, n),    # Remove
                   editDistance(str1, str2, m-1, n-1)    # Replace
                   )


def levenshtein(s1, s2):
   #code from: https://www.geeksforgeeks.org/edit-distance-dp-5/
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

#make pairs from annotations and predictions
def make_y_and_pred_pairs(y, predictions):
  y_pred_pairs = [] #index 0 - annotation, index 1-list of predictions
  has_true_y_on_index = []#index 0 True/False, index 1 true or closest prediction in y_pred_pairs[1]
  closest_prediction_to_true = []
  if predictions == []:
      predictions = [(0,'dummy')]
  for i in range(len(y)):
    y_pred_pairs.append([[],[]])
    has_true_y_on_index.append([False,0])
    closest_prediction_to_true.append(float('inf'))

  #phase 1, intervals between annotaions to half, each annotation get predictions from his two halfs
  for i in range(len(y)):
    y_pred_pairs[i][0] = y[i]
    tmp = []
    if i == 0:
      left = predictions[0][0]
      right = (y[i][0] + y[i+1][0]) / 2 
      for prediction in predictions:
        if prediction[0] >= left and  prediction[0] < right:
          y_pred_pairs[i][1].append(prediction)

          if prediction[1] == y[i][1]:
            if has_true_y_on_index[i][0] == False:
              has_true_y_on_index[i] = [True, len(y_pred_pairs[i][1]) - 1]
            elif has_true_y_on_index[i][0] == True and abs(y[i][0] - y_pred_pairs[i][1][has_true_y_on_index[i][1]][0]) > abs(y[i][0] - prediction[0]):
              has_true_y_on_index[i] = [True, len(y_pred_pairs[i][1]) - 1]

          if closest_prediction_to_true[i] > abs(y[i][0] - prediction[0]):
            closest_prediction_to_true[i] = abs(y[i][0] - prediction[0])


    elif i == (len(y)-1):
      left = (y[i-1][0] + y[i][0]) / 2
      right = predictions[-1][0]
      for prediction in predictions:
        if prediction[0] >= left and  prediction[0] <= right:
          y_pred_pairs[i][1].append(prediction)

          if prediction[1] == y[i][1]:
            if has_true_y_on_index[i][0] == False:
              has_true_y_on_index[i] = [True, len(y_pred_pairs[i][1]) - 1]
            elif has_true_y_on_index[i][0] == True and abs(y[i][0] - y_pred_pairs[i][1][has_true_y_on_index[i][1]][0]) > abs(y[i][0] - prediction[0]):
              has_true_y_on_index[i] = [True, len(y_pred_pairs[i][1]) - 1]

          if closest_prediction_to_true[i] > abs(y[i][0] - prediction[0]):
            closest_prediction_to_true[i] = abs(y[i][0] - prediction[0])

    else:
      left = (y[i-1][0] + y[i][0]) / 2
      right =(y[i][0] + y[i+1][0]) / 2 
      for prediction in predictions:
        if prediction[0] >= left and  prediction[0] < right:
          y_pred_pairs[i][1].append(prediction)

          if prediction[1] == y[i][1]:
            if has_true_y_on_index[i][0] == False:
              has_true_y_on_index[i] = [True, len(y_pred_pairs[i][1]) - 1]
            elif has_true_y_on_index[i][0] == True and abs(y[i][0] - y_pred_pairs[i][1][has_true_y_on_index[i][1]][0]) > abs(y[i][0] - prediction[0]):
              has_true_y_on_index[i] = [True, len(y_pred_pairs[i][1]) - 1]
          #print(f"i {i} should be putting")
          if closest_prediction_to_true[i] > abs(y[i][0] - prediction[0]):
            closest_prediction_to_true[i] = abs(y[i][0] - prediction[0])



  #phase 2
  for i in range(len(y)):
    if has_true_y_on_index[i][0] == False:
      #sentinel if annotation has TP predictions from phase1, if not TP then closest FP
      found_true_in_prev = False
      found_true_in_prev_id = 0
      found_true_in_next = False
      found_true_in_next_id = 0
      if i == 0:
        pass
      elif i == (len(y)-1):
        pass
      else:
        #pred sebe, jakoze i-1
        if len(y_pred_pairs[i-1][1]) > 1:#annotation before me has more than one label
          if has_true_y_on_index[i-1][0] == True: #has TP prediction neighbor before me
            sentinel = has_true_y_on_index[i-1][1] #sentinel is true label
          else: #neighbor before me doesnt has TP
            sentinel = closest_prediction_to_true[i-1] #sentinel is closest FP
          
          #try to find TP for me in neighbor before me
          for j in range(len(y_pred_pairs[i-1][1])-1,sentinel,-1): 
            if y_pred_pairs[i-1][1][j][1] == y[i][1] and y_pred_pairs[i-1][1][j][0] > y[i-1][0]: 
              found_true_in_prev = True
              found_true_in_prev_id = j

        
        if len(y_pred_pairs[i+1][1]) > 1:#same for negihbor after me
          if has_true_y_on_index[i+1][0] == True: 
            sentinel = has_true_y_on_index[i+1][1] l
          else: #nema true label
            sentinel = closest_prediction_to_true[i+1]

          for j in range(0,sentinel): 
            if y_pred_pairs[i+1][1][j][1] == y[i][1] and y_pred_pairs[i+1][1][j][0] < y[i+1][0]: 
              found_true_in_next = True
              found_true_in_next_id = j
        
        #if foudn TP for me in neighbor, take it
        if found_true_in_next == True and found_true_in_prev == True:
          pass
        elif found_true_in_prev == True: #
          for k in range(len(y_pred_pairs[i-1][1])-1,found_true_in_prev_id-1,-1):
             y_pred_pairs[i][1].insert(0,y_pred_pairs[i-1][1][k])

             if y_pred_pairs[i-1][1][k][1] == y[i][1]:
              if has_true_y_on_index[i][0] == False:
                has_true_y_on_index[i] = [True, k-found_true_in_prev_id] 
              elif has_true_y_on_index[i][0] == True and abs(y[i][0] - y_pred_pairs[i][1][has_true_y_on_index[i][1]+1][0]) > abs(y[i][0] - y_pred_pairs[i-1][1][k][1]): 
                has_true_y_on_index[i] = [True, k-found_true_in_prev_id]
              else:
                has_true_y_on_index[i][1] = has_true_y_on_index[i][1] + 1
             y_pred_pairs[i-1][1].pop(k)
             ############
        elif found_true_in_next == True: 
          for k in range(0,found_true_in_next_id+1):
             y_pred_pairs[i][1].append(y_pred_pairs[i+1][1][0]) 

             if y_pred_pairs[i+1][1][k][1] == y[i][1]:
              if has_true_y_on_index[i][0] == False:
                has_true_y_on_index[i] = [True, len(y_pred_pairs[i][1]) - 1] 
              elif has_true_y_on_index[i][0] == True and abs(y[i][0] - y_pred_pairs[i][1][has_true_y_on_index[i][1]][0]) > abs(y[i][0] - y_pred_pairs[i+1][1][k][1]): 
                has_true_y_on_index[i] = [True, len(y_pred_pairs[i][1]) - 1]
             y_pred_pairs[i+1][1].pop(0)

  return y_pred_pairs    



def evaluate_sequence(pairs, results_dict):
  #####################
  last_anotated = 0
  cnt = 0
  for pair in pairs:
    if len(pair[1]) != 0:
      last_anotated = max(last_anotated,cnt)
    cnt = cnt + 1
  #####################
  cnt = -1
  for pair in pairs:
    cnt = cnt + 1
    true_y = pair[0][1]
    true_frame = pair[0][0]
    correct_y_predictions = []


    for prediction in pair[1]:
      
      if prediction[1] == true_y:
        correct_y_predictions.append(prediction)
      else:
        results_dict[prediction[1]]['FP']['count'] =  results_dict[prediction[1]]['FP']['count'] + 1
        results_dict[prediction[1]]['FP']['dist_arr'].append(abs(true_frame - prediction[0]))
        if prediction[1] == 'SLNO':
          print("SLNO FP")

    

    #from more TP find closest one, other ones will be FP
    if len(correct_y_predictions) == 1:
      dist_from_true_frame = abs(true_frame - correct_y_predictions[0][0])
      results_dict[true_y]['TP']['count'] =  results_dict[true_y]['TP']['count'] + 1
      results_dict[true_y]['TP']['dist_arr'].append(dist_from_true_frame)
    elif len(correct_y_predictions) > 1: #
      distances = []
      for prediction in correct_y_predictions: 
        distances.append(abs(true_frame - prediction[0]))
      
      minnimum = min(distances)
      results_dict[true_y]['TP']['count'] =  results_dict[true_y]['TP']['count'] + 1
      results_dict[true_y]['TP']['dist_arr'].append(minnimum)
      results_dict[true_y]['FP']['count'] =  results_dict[true_y]['FP']['count'] + (len(correct_y_predictions) - 1)
      if prediction[1] == 'SLNO' and len(correct_y_predictions) > 1:
          print(f"SLNO FP {(len(correct_y_predictions) - 1)}")
      id_of_min = distances.index(min(distances))
      distances.pop(id_of_min)
      for dist in distances:
        results_dict[true_y]['FP']['dist_arr'].append(dist)

    elif len(correct_y_predictions) == 0:
      if not(pair[0][2] == 'after_end' and len(pair[1]) == 0 and cnt > last_anotated): #after end annotation handling
        results_dict[true_y]['FN']['count'] =  results_dict[true_y]['FN']['count'] + 1

  return results_dict

def read_predictions(path_to_load_events):
  predictions = []
  with open(path_to_load_events) as f:
    while True:
      line = f.readline()
      if line == '':
          break
      words = line.split(',')
      predictions.append(words[0],words[1])
  return predictions


def evaluate_dataset(frame_number1_lst, frame_number2_lst, path_to_results, true_annotations, results_dict):
  for frame_number1, frame_number2 in zip(frame_number1_lst,frame_number2_lst):
    path_to_load_events = path_to_results + 'events_' + str(frame_number1) + '_' +str(frame_number2)
    isExist =  os.path.exists(path_to_load_events)
    if isExist == False:
      print(f"{frame_number1}_{frame_number2} not found")
      continue
    #######if saved by joblib##########
    predictions = load(path_to_load_events)
    #if frame of event relative to start of game then add frame_number1
    for i in range(len(predictions)):
      predictions[i] = [predictions[i][0] + frame_number1,predictions[i][1]]
    ###################
    #if predictions saved to csv file
    #predictions = read_predictions(path_to_load_events)



    y_true = true_annotations.loc[np.logical_and(frame_number2 >= true_annotations.index, true_annotations.index >= frame_number1)] #vyber eventu mezi frame numbers
    y_true_lst = []
    for index, row in y_true.iterrows():
      y_true_lst.append([index,row['event1'],row['event2']])

    #make agragations by translate_anotations
    #y_true_lst =   translate_anotations(y_true_lst)
    #predictions = translate_anotations(predictions)
    print(f"{frame_number1}_{frame_number2}")
    print(f"y_true_lst {y_true_lst}")
    print(f"predictions {predictions}")
    pairs = make_y_and_pred_pairs(y_true_lst, predictions)
    results_dict = evaluate_sequence(pairs, results_dict)
  return results_dict  


def translate_anotations(to_translate):
  #set agragations if wanted
  for i in range(len(to_translate)):
    event = to_translate[i][1]
    if event == 'HFR' or event == 'HFL': 
      #event = 'hit'
      event = 'hit_far'
      #string_predictions.pop(i)
      pass

    elif event == 'HNR' or event == 'HNL':  
      #event = 'hit'
      event = 'hit_near'
      #string_predictions.pop(i)
      pass
    elif event == 'LFI' or event == 'LFO' or\
      event == 'SLFI' or event == 'SLFO':
      #event = 'land'
      event = 'land_far'
      #string_predictions.pop(i)
      pass
    elif event == 'LNI' or event == 'LNO' or\
    event == 'SLNI' or event == 'SLNO':
      #event = 'land'
      event = 'land_near'
      #string_predictions.pop(i)
      pass

    elif event == 'SFR' or event == 'SFL':
      event = 'serve_far'
      #string_predictions.pop(i)
      pass

    elif event == 'SNR' or event == 'SNL':
      event = 'serve_near'
      #string_predictions.pop(i)
      pass

    elif event == 'LNN' or event == 'LNF':
      #string_predictions.pop(i)
      pass
    to_translate[i][1] = event 
  return to_translate


def evaluate_dataset_edit_distance(frame_number1_lst, frame_number2_lst, path_to_results, true_annotations,results_dict):
  for frame_number1, frame_number2 in zip(frame_number1_lst,frame_number2_lst):
    path_to_load_events = path_to_results + 'events_' + str(frame_number1) + '_' +str(frame_number2)
    isExist =  os.path.exists(path_to_load_events)
    if isExist == False:
      print(f"{frame_number1}_{frame_number2} not found ------------------------------------------------------------------------ ------------------------------------------------------------------------ ------------------------------------------------------------------------")
      continue


    #######if saved by joblib##########
    predictions = load(path_to_load_events)
    #if predictions saved to csv file
    #predictions = read_predictions(path_to_load_events)


    string_predictions = []
    for i in range(len(predictions)):
      string_predictions.append(predictions[i][1])




    y_true = true_annotations.loc[np.logical_and(frame_number2 >= true_annotations.index, true_annotations.index >= frame_number1)] #vyber eventu mezi frame numbers
    
    string_y_true_tmp = []
    strings_y_true = []
  
    print(f"frame_number1 {frame_number1} frame_number2 {frame_number2} ------------------------------------------------------------------------")    

    num_edits = 1000000
    len_prediction = 0
    for index, row in y_true.iterrows():
      if row['event2'] != 'after_end':
        string_y_true_tmp.append(row['event1'])
      else:
        print(f"appending {string_y_true_tmp}")
        to_append = copy.deepcopy(string_y_true_tmp)
        strings_y_true.append(to_append)
        string_y_true_tmp.append(row['event1'])

    strings_y_true.append(string_y_true_tmp)

    if len(strings_y_true) > 1:
      print(f"strings {strings_y_true}")
    #set agragations if wanted, must be done same for   string_predictions and string_y_true
    for string_y_true in strings_y_true:
      for i in range(len(string_predictions)-1,-1,-1):
        if string_predictions[i] == 'HFR' or string_predictions[i] == 'HFL': 
          #string_predictions[i] = 'hit'
          #string_predictions[i] = 'hit_far'
          #string_predictions.pop(i)
          pass

        elif string_predictions[i] == 'HNR' or string_predictions[i] == 'HNL':  
          #string_predictions[i] = 'hit'
          #string_predictions[i] = 'hit_near'
          #string_predictions.pop(i)
          pass
        elif string_predictions[i] == 'LFI' or string_predictions[i] == 'LFO' or\
         string_predictions[i] == 'SLFI' or string_predictions[i] == 'SLFO':
          #string_predictions[i] = 'land'
          #string_predictions[i] = 'land_far'
          #string_predictions.pop(i)
          pass
        elif string_predictions[i] == 'LNI' or string_predictions[i] == 'LNO' or\
        string_predictions[i] == 'SLNI' or string_predictions[i] == 'SLNO':
          #string_predictions[i] = 'land'
          #string_predictions[i] = 'land_near'
          #string_predictions.pop(i)
          pass

        elif string_predictions[i] == 'SFR' or string_predictions[i] == 'SFL':
          #string_predictions[i] = 'serve_far'
          #string_predictions.pop(i)
          pass

        elif string_predictions[i] == 'SNR' or string_predictions[i] == 'SNL':
          #string_predictions[i] = 'serve_near'
          #string_predictions.pop(i)
          pass

        elif string_predictions[i] == 'LNN' or string_predictions[i] == 'LNF':
          #string_predictions.pop(i)
          pass

      for i in range(len(string_y_true)-1,-1,-1):
        if string_y_true[i] == 'HFR' or string_y_true[i] == 'HFL':
          #string_y_true[i] = 'hit'
          #string_y_true[i] = 'hit_far'
          #string_y_true.pop(i)
          pass
        elif string_y_true[i] == 'HNR' or string_y_true[i] == 'HNL':
          #string_y_true[i] = 'hit'
          #string_y_true[i] = 'hit_near'
          #string_y_true.pop(i)
          pass
        elif string_y_true[i] == 'LFI' or string_y_true[i] == 'LFO' or\
        string_y_true[i] == 'SLFI' or string_y_true[i] == 'SLFO':
          #string_y_true[i] = 'land'
          #string_y_true[i] = 'land_far'
          #string_y_true.pop(i)
          pass

        elif string_y_true[i] == 'LNI' or string_y_true[i] == 'LNO' or\
        string_y_true[i] == 'SLNI' or string_y_true[i] == 'SLNO':
          #string_y_true[i] = 'land'
          #string_y_true[i] = 'land_near'
          #string_y_true.pop(i)
          pass
        elif string_y_true[i] == 'SFR' or string_y_true[i] == 'SFL':
          #string_y_true[i] = 'serve_far'
          #string_y_true.pop(i)
          pass
        elif string_y_true[i] == 'SNR' or string_y_true[i] == 'SNL':
          #string_y_true[i] = 'serve_near'
          #string_y_true.pop(i)
          pass
        elif string_y_true[i] == 'LNN' or string_y_true[i] == 'LNF':
          #string_y_true.pop(i)
          pass
      print(f"true {string_y_true}")
      print(f"pred {string_predictions}")
      #num_edits = editDistance(string_predictions, string_y_true, len(string_predictions), len(string_y_true))
      num_edits_tmp = levenshtein(string_predictions, string_y_true)
      if num_edits > num_edits_tmp:
        num_edits = num_edits_tmp
      print(f"edit distance {num_edits_tmp}")
    print(f"min edit distance {num_edits}")
    len_y_true = len(string_y_true_tmp)
    results_dict['length_true'] = results_dict['length_true'] + len_y_true
    results_dict['length_predict'] = results_dict['length_predict'] + len(string_predictions)
    results_dict['num_edits'] = results_dict['num_edits'] + num_edits
    results_dict['lenghts_true'].append(len_y_true)
    results_dict['lenghts_pred'].append(len(string_predictions))
    results_dict['dists'].append(num_edits)


  return results_dict


def prepare_dataset(df, translate_file):
  #translate from numbers to strings
  df_translated = copy.deepcopy(df)
  for index, row in df.iterrows():
    print(f"index {index} row {row['event1']} row {row['event2']}")
    event1_translate = translate_file.iloc[int(row['event1'])]['events']
    event2_translate = ''
    if not math.isnan(row['event2']):
      event2_translate = translate_file.iloc[int(row['event1'])]['events']
      if event1_translate == 'after_end':
        df_translated.loc[df_translated.index == index, 'event1'] = translate_file.iloc[int(row['event2'])]['events']
        df_translated.loc[df_translated.index == index, 'event2'] = translate_file.iloc[int(row['event1'])]['events']
      else:
        df_translated.loc[df_translated.index == index, 'event1'] = translate_file.iloc[int(row['event1'])]['events']
        df_translated.loc[df_translated.index == index, 'event2'] = translate_file.iloc[int(row['event2'])]['events']
    else:
      df_translated.loc[df_translated.index == index, 'event1'] = translate_file.iloc[int(row['event1'])]['events']

  return df_translated



def get_serve_speed_error(serve_speed_lst, frame_number1_lst, frame_number2_lst, path_to_results):
  MAPE = 0
  MAE = 0
  cnt = 0
  comparsions_cnt = 0

  true_speed_lst1 = []
  prediction_lst1 = []
  true_speed_lst2 = []
  prediction_lst2 = []
  true_speed_lst3 = []
  prediction_lst3 = []
  error_lst = []
  anotations_cnt = 0
  for frame_number1, frame_number2 in zip(frame_number1_lst,frame_number2_lst):
    path_to_load_events = path_to_results + 'serve_speed_' + str(frame_number1) + '_' +str(frame_number2)
    isExist =  os.path.exists(path_to_load_events)
    #print(f"frame1 {frame_number1} frame2 {frame_number2}")
    
    
    if isExist == False or serve_speed_lst[cnt] =='x':
      print(f"isExist {isExist} vs serve_speed_lst[cnt] {serve_speed_lst[cnt]}")
      cnt = cnt + 1
      continue
    prediction = load(path_to_load_events)
    if prediction !='x':
      anotations_cnt = anotations_cnt +1
    if prediction !='x':
      
      #print(f"prediction {prediction} vs serve_speed_lst[cnt] {serve_speed_lst[cnt]}")

      if abs(float(prediction) - float(serve_speed_lst[cnt])) < 70: #set max error for serve speed to put out outliers
        MAE = MAE + abs(float(prediction) - float(serve_speed_lst[cnt]))
        #print(f"prediction {abs(float(prediction) - float(serve_speed_lst[cnt]))}")
        MAPE = MAPE + (abs(float(prediction) - float(serve_speed_lst[cnt])) / float(serve_speed_lst[cnt]))*100
        comparsions_cnt = comparsions_cnt + 1
        true_speed_lst1.append(float(serve_speed_lst[cnt]))
        prediction_lst1.append(float(prediction))
        true_speed_lst3.append(float(serve_speed_lst[cnt]))
        prediction_lst3.append(float(prediction))
        #error_lst.append(abs(float(prediction) - float(serve_speed_lst[cnt])))
        error_lst.append( (abs(float(prediction) - float(serve_speed_lst[cnt])) /float(serve_speed_lst[cnt]))*100)
      else:
        print(f"error too big {frame_number1} {frame_number2}")
        print(f"prediction {prediction} vs serve_speed_lst[cnt] {serve_speed_lst[cnt]} error {abs(float(prediction) - float(serve_speed_lst[cnt]))}")
        true_speed_lst2.append(float(serve_speed_lst[cnt]))
        prediction_lst2.append(float(prediction))
        true_speed_lst3.append(float(serve_speed_lst[cnt]))
        prediction_lst3.append(float(prediction))

    cnt = cnt + 1    
  MAE = MAE / comparsions_cnt
  MAPE = MAPE / comparsions_cnt
  return MAE,MAPE,true_speed_lst1,prediction_lst1,error_lst,anotations_cnt,comparsions_cnt,true_speed_lst2,prediction_lst2,true_speed_lst3,prediction_lst3      

In [10]:
video_name ='V008'
#dataset files
path_to_annotations = '/content/drive/My Drive/tenis/videos/' + video_name + '/default.txt'
path_to_translate_file = '/content/drive/My Drive/tenis/videos/' + video_name + '/synsets.txt'
true_annotations = pd.read_csv(path_to_annotations, delimiter=r"\s+",names=['frame','event1','event2'])
translate_file = pd.read_csv(path_to_translate_file, delimiter=r"\s+", names=['events'])


path_to_load_clips_sequences = '/content/drive/My Drive/tenis/videos/' + video_name + '/V008_annotation_clips.csv'
clips_sequences = pd.read_csv(path_to_load_clips_sequences, delimiter=r",")


frame_number1_lst = []
frame_number2_lst = []
serve_speed_lst = []
for index, row in clips_sequences.iterrows():
  frame_number1_lst.append(row[0])
  frame_number2_lst.append(row[1])
  serve_speed_lst.append(row[2])

path_to_results = '/content/drive/My Drive/tenis/videos/' + video_name + '/results10/'


#results6/serve_speed_' + str(frame_number1) + '_' +str(frame_number2)
true_annotations = true_annotations[~true_annotations['event1'].isnull()]
true_annotations = prepare_dataset(true_annotations, translate_file)

###############for TP,TN,FP#####################
results_dict = get_results_dict()
#results_dict = evaluate_dataset(frame_number1_lst, frame_number2_lst, path_to_results, true_annotations, results_dict)
###############################################

################ for edit distance#################
results_dict = {'length_true':0, 'length_predict':0, 'num_edits':0,'lenghts_true':[],'lenghts_pred':[], 'dists':[] }
results_dict = evaluate_dataset_edit_distance(frame_number1_lst, frame_number2_lst, path_to_results, true_annotations,results_dict)
###########################################

######################count annotations in dataset##########
count_dict = get_count_dict()
##########################################################

################ for speed serve evaluations#################    
#MAE,MAPE,true_speed_lst1,prediction_lst1,error_lst,anotations_cnt,comparsions_cnt,true_speed_lst2,prediction_lst2, true_speed_lst3,prediction_lst3= get_serve_speed_error(serve_speed_lst, frame_number1_lst, frame_number2_lst, path_to_results)
###########################################
  

index 1984 row 12.0 row nan
index 1996 row 14.0 row nan
index 2003 row 1.0 row nan
index 2023 row 8.0 row nan
index 2032 row 4.0 row nan
index 2055 row 6.0 row nan
index 2067 row 0.0 row nan
index 2087 row 8.0 row nan
index 2100 row 3.0 row nan
index 2128 row 6.0 row nan
index 2135 row 0.0 row nan
index 2156 row 8.0 row nan
index 2758 row 12.0 row nan
index 2769 row 14.0 row nan
index 2777 row 1.0 row nan
index 2793 row 8.0 row nan
index 2803 row 4.0 row nan
index 2861 row 7.0 row nan
index 2901 row 18.0 row 7.0
index 3400 row 13.0 row nan
index 3413 row 14.0 row nan
index 3421 row 0.0 row nan
index 3452 row 8.0 row nan
index 3462 row 4.0 row nan
index 3484 row 6.0 row nan
index 3497 row 1.0 row nan
index 3517 row 8.0 row nan
index 4055 row 12.0 row nan
index 4067 row 14.0 row nan
index 4074 row 0.0 row nan
index 4103 row 9.0 row nan
index 4122 row 18.0 row 4.0
index 4145 row 18.0 row 6.0
index 4164 row 18.0 row 7.0
index 4655 row 13.0 row nan
index 4666 row 14.0 row nan
index 4675 row

In [ ]:
#count annotations

for index, row in true_annotations.iterrows():
    count_dict[row['event1']]['count'] = count_dict[row['event1']]['count'] + 1
    if row['event2'] == 'after_end':
      count_dict[row['event1']]['after_end'] = count_dict[row['event1']]['after_end'] + 1


count_dict = get_count_dict()

for index, row in true_annotations.iterrows():
    count_dict[row['event1']]['count'] = count_dict[row['event1']]['count'] + 1
    if row['event2'] == 'after_end':
      count_dict[row['event1']]['after_end'] = count_dict[row['event1']]['after_end'] + 1
    if row['event2'] == 'after_end' and row['event1'] == 'SLFI':
      print(f"{row}")

In [ ]:
#get ED accuracy
print(f"{results_dict['length_predict']} {results_dict['num_edits']}")
((results_dict['length_predict'] - results_dict['num_edits']) / results_dict['length_predict']) *100 #accuracy

In [ ]:
#get TP FP FN, recall, accuracy, F1
for key in results_dict:
  
  if key == 'after_end' or key == 'dummy' or key == 'let':
    continue
  if (results_dict[key]['TP']['count'] + results_dict[key]['FN']['count']) != 0:
    print(f"{key}")
    recall = (results_dict[key]['TP']['count'] / (results_dict[key]['TP']['count'] + results_dict[key]['FN']['count']))*100
    precision = (results_dict[key]['TP']['count'] / (results_dict[key]['TP']['count'] + results_dict[key]['FP']['count']))*100

    f1 = (2 * results_dict[key]['TP']['count'] / (2*results_dict[key]['TP']['count'] + results_dict[key]['FP']['count'] + results_dict[key]['FN']['count']))*100

    print(f"{key}: TP {results_dict[key]['TP']['count']}, FP {results_dict[key]['FP']['count']}, FN {results_dict[key]['FN']['count']}")
    print(f"{key}: recall {recall} precision {precision} f1 {f1}")
  else:
    #print(f"skipping {key}")
    pass

In [ ]:
#box plots of distances
red_circle = dict(markerfacecolor='red', marker='o', markeredgecolor='white')

fig, axs = plt.subplots(1, len(df.columns), figsize=(20,10))

for i, ax in enumerate(axs.flat):
    ax.boxplot(df.iloc[:,i], flierprops=red_circle)
    ax.set_title(df.columns[i], fontsize=20, fontweight='bold')
    ax.tick_params(axis='y', labelsize=14)
    
plt.tight_layout()

dists = {'serve_far':results_dict['serve_far']['TP']['dist_arr'],
         'serve_near':results_dict['serve_near']['TP']['dist_arr'],
         'land_far':results_dict['land_far']['TP']['dist_arr'],
         'land_near':results_dict['land_near']['TP']['dist_arr'],
         'hit_far':results_dict['hit_far']['TP']['dist_arr'],
         'hit_near':results_dict['hit_near']['TP']['dist_arr'],
         'LNN':results_dict['LNN']['TP']['dist_arr'],
         'LNF':results_dict['LNF']['TP']['dist_arr'],
}

from matplotlib.pyplot import figure
import matplotlib.ticker as ticker
fig, ax = plt.subplots(figsize=(10,10))
bp = ax.boxplot(dists.values(),vert=False, )
plt.setp(bp['boxes'], color='blue', linewidth=4)
plt.setp(bp['whiskers'], color='blue', linewidth=4)
plt.setp(bp['fliers'], color='blue', marker='o', markerfacecolor='r', markersize=8)
plt.setp(bp['medians'], color='darkorange', linewidth=4)
plt.setp(bp['caps'], color='red', linewidth=4)

plt.grid(axis = 'x',linewidth=2)
#for element in ['boxes', 'whiskers', 'fliers', 'means', 'medians', 'caps']:
tick_spacing = 1
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
#ax.grid(linewidth=2)
ax.tick_params(direction='out', width=4,)
ax.set_yticklabels(dists.keys())
plt.xticks(fontsize= 14,)
plt.yticks(fontsize= 14,)
plt.xlabel('vzdálenost v počtu snímků mezi predikcí a anotací',fontsize=15,)

In [ ]:
#delete events with distance bigger than 5
for key in results_dict:
  
  if key == 'after_end' or key == 'dummy' or key == 'let':
    continue
  if (results_dict[key]['TP']['count'] + results_dict[key]['FN']['count']) != 0:
    print(f"{key}")
    for i in range(len(results_dict[key]['TP']['dist_arr'])-1,-1,-1):
      if results_dict[key]['TP']['dist_arr'][i] > 5:
        results_dict[key]['TP']['dist_arr'].pop(i)
        results_dict[key]['TP']['count'] = results_dict[key]['TP']['count'] - 1
        results_dict[key]['FP']['count'] = results_dict[key]['FP']['count'] + 1

    recall = (results_dict[key]['TP']['count'] / (results_dict[key]['TP']['count'] + results_dict[key]['FN']['count']))*100
    precision = (results_dict[key]['TP']['count'] / (results_dict[key]['TP']['count'] + results_dict[key]['FP']['count']))*100

    f1 = (2 * results_dict[key]['TP']['count'] / (2*results_dict[key]['TP']['count'] + results_dict[key]['FP']['count'] + results_dict[key]['FN']['count']))*100

    print(f"{key}: TP {results_dict[key]['TP']['count']}, FP {results_dict[key]['FP']['count']}, FN {results_dict[key]['FN']['count']}")
    print(f"{key}: recall {recall} precision {precision} f1 {f1}")
  else:
    #print(f"skipping {key}")
    pass